# Functions that are auxiliary to the training process

In [ ]:
import numpy as np
import glob
import imageio
import time
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from PIL import Image
%matplotlib inline

In [ ]:
def get_noisy_binary_labels(batch_size, label, label_range):
    if label:
        # Values close to 1.0 no intervalo [1 - label_range, 1)
        random_samples = (np.ones(batch_size) - (np.random.random_sample(batch_size) * label_range))
    else:
        # Values close to 0.0 no intervalo [0, label_range)
        random_samples = np.random.random_sample(batch_size) * label_range
    
    return random_samples

In [ ]:
# Transform values from [0,255] to the range [-1.0,1.0] 
def norm_img(img):
    img = (img / 127.5) - 1
    return img

In [ ]:
# Transform values from [-1.0,1.0] to the range [0,255]
def denorm_img(img):
    img = (img + 1) * 127.5
    return img.astype(np.uint8)

In [ ]:
# Get image samples from the dataset
def sample_from_dataset(batch_size, image_shape, data_dir=None, data = None):
    
    # Create tensor 
    sample_dim = (batch_size,) + image_shape
    sample = np.empty(sample_dim, dtype=np.float32)
    
    # Choose image names from the data_dir
    all_data_dirlist = list(glob.glob(data_dir))
    sample_imgs_paths = np.random.choice(all_data_dirlist, batch_size)
    
    # Read and preprocess images from the dataset
    for index, img_filename in enumerate(sample_imgs_paths):
        image = Image.open(img_filename)
        image = image.resize(image_shape[:-1])
        image = image.convert('RGB') #remove transparent ('A') layer
        image = np.asarray(image)
        plt.imshow(np.asarray(image))
        image = norm_img(image)
        sample[index,...] = image
    return sample

### Generate input noise $Z$ to the Generator $G$

NOTE: Using noise produced by __np.random.uniform__,the generator seems to produce _same image_ for _ANY noise_,  
but those images (even though they are the same) are very close to the actual image - experiment with it later.

In [ ]:
def get_normal_noise_vector(batch_size, noise_shape):
    return np.random.normal(0, 1, size = (batch_size,) + noise_shape)

### Define function to generate images from a trained generator $G$

In [ ]:
def generate_image(generator, noise_shape, save_dir):
    
    noise = get_normal_noise_vector(1, noise_shape)
    fake_data_X = generator.predict(noise)
    array_img = denorm_img(fake_data_X[0, :,:,:])

    im = Image.fromarray(array_img)
    
    # Save generate image
    fname = save_dir +'/'+ str(time.time()) + "_fake_image.png"
    im.save(fname)


### Save Image Batch 

In [ ]:
def save_img_batch(img_batch, img_save_dir):
    nr = 4      # number of rows of the grid
    nc = 4      # number of columns of the grid    
    nig = nr*nc # number of images inside the grid
    
    plt.figure(figsize=(nr, nc))
    gs1 = gridspec.GridSpec(nr, nc)
    gs1.update(wspace = 0, hspace = 0)
    rand_indices = np.random.choice(img_batch.shape[0], nig, replace = False)
    #print(rand_indices)
    for i in range(nig):
        #plt.subplot(nr, nc, i+1)
        ax1 = plt.subplot(gs1[i])
        ax1.set_aspect('equal')
        rand_index = rand_indices[i]
        image = img_batch[rand_index, :,:,:]
        fig = plt.imshow(denorm_img(image))
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
    plt.tight_layout()
    plt.savefig(img_save_dir, bbox_inches = 'tight', pad_inches = 0)
    plt.show()  

In [ ]:
def save_GIF_movie_from_PNG(img_save_dir, name):
    images = []
    all_data_dirlist = list(glob.glob(img_save_dir + "*_image.png"))
    
    for filename in all_data_dirlist:
        img_num = filename.split('/')[-1][0:-10]
        if (int(img_num) % 100) == 0:
            images.append(imageio.imread(filename))
            
    imageio.mimsave(img_save_dir + name + '_movie.gif', images) 

In [ ]:
def print_average_loss(RD_loss, FD_loss, GAN_loss):
        print("-----------------------------------------------------------------")
        print("Average (Real) D loss: %f" % (np.mean(RD_loss)))    
        print("Average (Fake) D loss: %f" % (np.mean(FD_loss)))    
        print("Average G loss: %f" % (np.mean(GAN_loss)))
        print("-----------------------------------------------------------------")
       